In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Generate a periodic time series for real data
t = np.linspace(0, 2 * np.pi, 100)
real_data = np.sin(t)

# Calculate max and min values at each time point (for illustration, we use a constant offset)
max_values = real_data + 0.5
min_values = real_data - 0.5

# Generate synthetic time series with similar periodicity
synthetic_data = np.sin(t + 0.2)  # Shifted sine wave for illustration

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(t, max_values, label='Max values', color='red')
plt.plot(t, min_values, label='Min values', color='blue')
plt.fill_between(t, min_values, max_values, color='gray', alpha=0.3)
plt.plot(t, synthetic_data, label='Synthetic data', color='orange')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Evaluation Metric for Synthetic Time Series')
plt.legend()
plt.show()


In [ ]:
def dynamic_time_warping_dist(X, Y, norm=np.linalg.norm):
    N = len(X)
    M = len(Y)
    D = np.zeros((N + 1, M + 1), dtype=float)
    D[:, 0] = np.inf
    D[0, :] = np.inf
    D[0][0] = 0

    for i in range(1, N + 1):
        for j in range(1, M + 1):
            m = min([D[i-1][j], D[i][j-1], D[i - 1][j - 1]])
            D[i][j] = norm(X[i - 1] - Y[j-1]) + m
    
    return D, D[N][M]


In [3]:
#from generator.acgan import Generator, Discriminator
from data_utils.dataset import PecanStreetDataset

In [1]:
import pandas as pd
from data_utils.dataset import PecanStreetDataset, prepare_dataloader, split_dataset

dataset = PecanStreetDataset().data
dataset = dataset[["dataid", "grid", "month", "day"]]
dataset


,dataid,grid,month,day
0,27,-1.837765,5,0
1,27,-1.726164,5,0
2,27,-1.492420,5,0
3,27,-1.196961,5,0
4,27,0.488622,5,0
...,...,...,...,...
441594,9053,-0.019480,10,6
441595,9053,0.030620,10,6
441596,9053,0.023463,10,6
441597,9053,0.066405,10,6


In [43]:
def preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize the 'grid' column in the data based on groupings of 'dataid', 'month', and 'day'.

    Args:
        data (pd.DataFrame): The input data with columns 'dataid', 'month', 'day', and 'grid'.

    Returns:
        pd.DataFrame: The normalized data.
    """
    if not all(col in data.columns for col in ['dataid', 'month', 'day', 'grid']):
        raise ValueError("Input data must contain 'dataid', 'month', 'day', and 'grid' columns")
    
    grouped = data.groupby(['dataid', 'month', 'day'])

    def normalize(group):
        mean = group['grid'].mean()
        std = group['grid'].std()
        group['grid'] = (group['grid'] - mean) / std
        return group

    normalized_data = grouped.apply(normalize)
    normalized_data = normalized_data.reset_index(drop=True)

    return normalized_data